# Sampling and Splitting

## Sampling

Sampling for basic tabular datasets. (Not designed for time series as of now.)

In [1]:
import polars as pl
import polars_ds as pds
import polars_ds.sample_and_splits as sa

In [2]:
df = pds.frame(size=100_000).with_columns(
    pds.random(0.0, 12.0).alias("uniform_1"),
    pds.random(0.0, 1.0).alias("uniform_2"),
    pds.random_exp(0.5).alias("exp"),
    pds.random_normal(0.0, 1.0).alias("normal"),
    pds.random_normal(0.0, 1000.0).alias("fat_normal"),
    (pds.random_int(0, 3)).alias("flags"),
    pl.Series(["A"] * 30_000 + ["B"] * 30_000 + ["C"] * 40_000).alias("category"),
)
df.head()

row_num,uniform_1,uniform_2,exp,normal,fat_normal,flags,category
i64,f64,f64,f64,f64,f64,i32,str
0,11.206726,0.894185,4.458632,-1.050551,-889.082412,2,"""A"""
1,10.582617,0.178725,0.30583,1.600861,-1314.229192,2,"""A"""
2,7.428565,0.179167,1.25681,1.542938,-508.690208,0,"""A"""
3,2.7745,0.952779,1.743263,0.53713,559.948959,0,"""A"""
4,9.308098,0.409356,0.451091,1.245293,395.857481,0,"""A"""


In [3]:
sa.random_cols(df, 2, keep = ["row_num"])

['row_num', 'normal', 'flags']

In [4]:
# Random Sample
sa.sample(df, 0.6) # by ratio

row_num,uniform_1,uniform_2,exp,normal,fat_normal,flags,category
i64,f64,f64,f64,f64,f64,i32,str
1,10.582617,0.178725,0.30583,1.600861,-1314.229192,2,"""A"""
2,7.428565,0.179167,1.25681,1.542938,-508.690208,0,"""A"""
3,2.7745,0.952779,1.743263,0.53713,559.948959,0,"""A"""
4,9.308098,0.409356,0.451091,1.245293,395.857481,0,"""A"""
5,0.197613,0.50962,1.487213,0.899812,-504.788795,0,"""A"""
…,…,…,…,…,…,…,…
99991,9.855049,0.550736,16.33884,-1.237213,428.289176,0,"""C"""
99992,10.678872,0.592427,0.884442,0.476622,-3040.243293,0,"""C"""
99995,1.596649,0.873083,3.649007,1.359436,-1275.453144,0,"""C"""


In [5]:
sa.sample(df, 30_000) # by count

row_num,uniform_1,uniform_2,exp,normal,fat_normal,flags,category
i64,f64,f64,f64,f64,f64,i32,str
0,11.206726,0.894185,4.458632,-1.050551,-889.082412,2,"""A"""
3,2.7745,0.952779,1.743263,0.53713,559.948959,0,"""A"""
6,10.429956,0.882234,2.800236,-0.468756,-358.755424,0,"""A"""
11,6.535157,0.655297,1.563761,0.425741,-140.830527,2,"""A"""
13,3.734648,0.603236,3.702746,-1.778182,-57.954508,0,"""A"""
…,…,…,…,…,…,…,…
99981,2.456735,0.453969,0.010071,-1.933908,-839.604284,0,"""C"""
99982,5.420465,0.316977,5.236476,-0.365679,826.449517,1,"""C"""
99983,6.719658,0.909437,1.238917,-0.297085,-601.853564,2,"""C"""


In [6]:
df.group_by("flags").len().sort("flags")

flags,len
i32,u32
0,33277
1,33463
2,33260


In [7]:
# Downsample on one group
sa1 = sa.downsample(
    df, 
    (pl.col("flags") == 0, 0.5)
)
sa1.group_by("flags").len().sort("flags")

flags,len
i32,u32
0,16638
1,33463
2,33260


In [8]:
# Downsample on multiple groups
sa2 = sa.downsample(
    df, 
    (pl.col("flags") == 0, 0.5),
    (pl.col("flags") == 1, 0.3),
    (pl.col("flags") == 2, 0.4),
)
sa2.group_by("flags").len().sort("flags")

flags,len
i32,u32
0,16638
1,10038
2,13304


In [9]:
df.group_by("category").len().sort("category")

category,len
str,u32
"""A""",30000
"""B""",30000
"""C""",40000


In [10]:
# Volume neutral by each category, will take the greatest possible value so that we get neutral volume.
vn = sa.volume_neutral(
    df,
    by = pl.col("category"),
)
vn.group_by("category").len().sort("category")

category,len
str,u32
"""A""",30000
"""B""",30000
"""C""",30000


In [11]:
# Volume neutral (10_000) by each category
vn = sa.volume_neutral(
    df,
    by = pl.col("category"),
    target_volume = 10_000
)
vn.group_by("category").len().sort("category")

category,len
str,u32
"""A""",10000
"""B""",10000
"""C""",10000


In [12]:
# Volume neutral (10_000) by a more complicated condition
vn = sa.volume_neutral(
    df,
    by = pl.col("category") == "A",
    target_volume = 10_000
) # This makes sense because count for B + count for C = 10_000
vn.group_by("category").len().sort("category")

category,len
str,u32
"""A""",10000
"""B""",4223
"""C""",5777


In [13]:
# Volume neutral sample with a control level. Volume neutral happens under the category level, meaning
# the volume for each flag in each category is neutral.
vn = sa.volume_neutral(
    df,
    by = pl.col("flags"),
    control = pl.col("category")
) 
vn.group_by(["category", "flags"]).len().sort(["category", "flags"])

category,flags,len
str,i32,u32
"""A""",0,9927
"""A""",1,9927
"""A""",2,9927
"""B""",0,9946
"""B""",1,9946
"""B""",2,9946
"""C""",0,13215
"""C""",1,13215
"""C""",2,13215


In [14]:
# We may not meet the target volume for all categories.
vn = sa.volume_neutral(
    df,
    by = pl.col("flags"),
    control = pl.col("category"),
    target_volume= 10_000
) # 
vn.group_by(["category", "flags"]).len().sort(["category", "flags"])

category,flags,len
str,i32,u32
"""A""",0,9927
"""A""",1,9927
"""A""",2,9927
"""B""",0,9946
"""B""",1,9946
"""B""",2,9946
"""C""",0,10000
"""C""",1,10000
"""C""",2,10000


## Splitting

Split by ratios.

In [15]:
print(df.shape)
train, test = sa.split_by_ratio(
    df,
    split_ratio = 0.6
)
print(train.shape)
print(test.shape)


(100000, 8)
(60000, 8)
(40000, 8)


In [16]:
print(df.shape)
for frame in sa.split_by_ratio(df, split_ratio = [0.25, 0.4, 0.10, 0.25]):
    print(frame.shape)

(100000, 8)
(25000, 8)
(40000, 8)
(10000, 8)
(25000, 8)
